## Configure training

The following os.environ lines can be customized to set the words that will be trained for, and the steps and learning rate of the training. The default values will result in the same model that is used in the micro_speech example. Run the cell to set the configuration:

In [1]:
import os

# A comma-delimited list of the words you want to train for.
# The options are: yes,no,up,down,left,right,on,off,stop,go
# All other words will be used to train an "unknown" category.
os.environ["WANTED_WORDS"] = "zero,one,two,three,four,five,six,seven,eight,nine"

# The number of steps and learning rates can be specified as comma-separated
# lists to define the rate at each stage. For example,
# TRAINING_STEPS=15000,3000 and LEARNING_RATE=0.001,0.0001
# will run 18,000 training loops in total, with a rate of 0.001 for the first
# 15,000, and 0.0001 for the final 3,000.
os.environ["TRAINING_STEPS"]="15000,3000"
os.environ["LEARNING_RATE"]="0.001,0.0001"

# Calculate the total number of steps, which is used to identify the checkpoint
# file name.
total_steps = sum(map(lambda string: int(string),
                  os.environ["TRAINING_STEPS"].split(",")))
os.environ["TOTAL_STEPS"] = str(total_steps)

# Print the configuration to confirm it
!echo "Training these words: ${WANTED_WORDS}"
!echo "Training steps in each stage: ${TRAINING_STEPS}"
!echo "Learning rate in each stage: ${LEARNING_RATE}"
!echo "Total number of training steps: ${TOTAL_STEPS}"

Training these words: zero,one,two,three,four,five,six,seven,eight,nine
Training steps in each stage: 15000,3000
Learning rate in each stage: 0.001,0.0001
Total number of training steps: 18000


## Install dependencies
Next, we'll install a GPU build of TensorFlow, so we can use GPU acceleration for training.

In [2]:
!pip install tensorflow-gpu==1.15

Uninstalling tensorflow-gpu-1.15.0:
  Successfully uninstalled tensorflow-gpu-1.15.0
Uninstalling tensorflow-estimator-1.15.1:
  Successfully uninstalled tensorflow-estimator-1.15.1
  Using cached https://files.pythonhosted.org/packages/a5/ad/933140e74973fb917a194ab814785e7c23680ca5dee6d663a509fe9579b6/tensorflow_gpu-1.15.0-cp36-cp36m-manylinux2010_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/de/62/2ee9cd74c9fa2fa450877847ba560b260f5d0fb70ee0595203082dafcc9d/tensorflow_estimator-1.15.1-py2.py3-none-any.whl


We'll also clone the TensorFlow repository, which contains the scripts that train and freeze the model.

In [3]:
# Clone the repository from GitHub
!git clone -q https://github.com/tensorflow/tensorflow

fatal: destination path 'tensorflow' already exists and is not an empty directory.


## Begin training
Next, run the following script to begin training. The script will first download the training data:

In [ ]:
!python ./tensorflow/tensorflow/examples/speech_commands/train.py \
--model_architecture=tiny_conv --window_stride=20 --preprocess=micro \
--wanted_words=${WANTED_WORDS} --silence_percentage=25 --unknown_percentage=25 \
--quantize=1 --verbosity=WARN --how_many_training_steps=${TRAINING_STEPS} \
--learning_rate=${LEARNING_RATE} --summaries_dir=./content/retrain_logs \
--data_dir=./content/speech_dataset --train_dir=./content/speech_commands_train \

2020-03-05 21:29:38.110603: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2020-03-05 21:29:38.120235: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2596985000 Hz
2020-03-05 21:29:38.122812: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55f57fd05410 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2020-03-05 21:29:38.122838: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2020-03-05 21:29:38.125335: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-03-05 21:29:38.726706: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55f57fdc2250 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2020-03-05 21:29:38.726767: I tensorflow/compiler/xla/s

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0305 21:29:46.802170 139670880511744 deprecation.py:323] From /anaconda/envs/azureml_py36/lib/python3.6/site-packages/tensorflow_core/python/ops/losses/losses_impl.py:121: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

W0305 21:29:46.810488 139670880511744 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://g

## Training finished

Export the model in a compact format

In [8]:
!python ./tensorflow/tensorflow/examples/speech_commands/freeze.py \
--model_architecture=tiny_conv --window_stride=20 --preprocess=micro \
--wanted_words=${WANTED_WORDS} --quantize=1 --output_file=./content/tiny_conv.pb \
--start_checkpoint=./content/speech_commands_train/tiny_conv.ckpt-${TOTAL_STEPS}

2020-03-09 18:57:01.667498: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2020-03-09 18:57:01.677774: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2596985000 Hz
2020-03-09 18:57:01.680670: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x5599f3be52a0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2020-03-09 18:57:01.680698: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2020-03-09 18:57:01.683212: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-03-09 18:57:02.260902: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x5599f3ca3610 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2020-03-09 18:57:02.260948: I tensorflow/compiler/xla/s

INFO:tensorflow:Saved frozen graph to ./content/tiny_conv.pb
I0309 18:57:04.262484 140484404909824 freeze.py:186] Saved frozen graph to ./content/tiny_conv.pb


## Convert the model

Run this cell to use the TensorFlow Lite converter to convert the frozen graph into the TensorFlow Lite format, fully quantized for use with embedded devices.

In [9]:
!toco \
--graph_def_file=./content/tiny_conv.pb --output_file=./content/tiny_conv.tflite \
--input_shapes=1,49,40,1 --input_arrays=Reshape_2 --output_arrays='labels_softmax' \
--inference_type=QUANTIZED_UINT8 --mean_values=0 --std_dev_values=9.8077

2020-03-09 18:58:19.728031: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-03-09 18:58:19.862304: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: Tesla K80 major: 3 minor: 7 memoryClockRate(GHz): 0.8235
pciBusID: 29d7:00:00.0
2020-03-09 18:58:19.863408: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 1 with properties: 
name: Tesla K80 major: 3 minor: 7 memoryClockRate(GHz): 0.8235
pciBusID: 4a56:00:00.0
2020-03-09 18:58:19.864460: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 2 with properties: 
name: Tesla K80 major: 3 minor: 7 memoryClockRate(GHz): 0.8235
pciBusID: 6753:00:00.0
2020-03-09 18:58:19.865511: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 3 with properties: 
name: Tesla K80 major: 3 minor: 7 memoryClockRate(GHz): 0.8235
pciBusID: 9186:00:00.0
2020-03-09 18:58:19.865767: I tensorflow/st

Print the model size

In [10]:
import os
model_size = os.path.getsize("./content/tiny_conv.tflite")
print("Model is %d bytes" % model_size)

Model is 50320 bytes


Finally, we use xxd to transform the model into a source file that can be included in a C++ project and loaded by TensorFlow Lite for Microcontrollers.

In [13]:
# Save the file as a C source file
!xxd -i ./content/tiny_conv.tflite > ./content/tiny_conv.cc
# Print the source file
!cat ./content/tiny_conv.cc

unsigned char __content_tiny_conv_tflite[] = {
  0x1c, 0x00, 0x00, 0x00, 0x54, 0x46, 0x4c, 0x33, 0x00, 0x00, 0x12, 0x00,
  0x1c, 0x00, 0x04, 0x00, 0x08, 0x00, 0x0c, 0x00, 0x10, 0x00, 0x14, 0x00,
  0x00, 0x00, 0x18, 0x00, 0x12, 0x00, 0x00, 0x00, 0x03, 0x00, 0x00, 0x00,
  0x3c, 0xc4, 0x00, 0x00, 0x10, 0x00, 0x00, 0x00, 0x1c, 0x00, 0x00, 0x00,
  0x2c, 0x00, 0x00, 0x00, 0x0c, 0x00, 0x00, 0x00, 0x01, 0x00, 0x00, 0x00,
  0xc0, 0x00, 0x00, 0x00, 0x01, 0x00, 0x00, 0x00, 0x94, 0x00, 0x00, 0x00,
  0x0f, 0x00, 0x00, 0x00, 0x54, 0x4f, 0x43, 0x4f, 0x20, 0x43, 0x6f, 0x6e,
  0x76, 0x65, 0x72, 0x74, 0x65, 0x64, 0x2e, 0x00, 0x0a, 0x00, 0x00, 0x00,
  0x60, 0x00, 0x00, 0x00, 0x54, 0x00, 0x00, 0x00, 0x48, 0x00, 0x00, 0x00,
  0x3c, 0x00, 0x00, 0x00, 0x34, 0x00, 0x00, 0x00, 0x2c, 0x00, 0x00, 0x00,
  0x20, 0x00, 0x00, 0x00, 0x18, 0x00, 0x00, 0x00, 0x10, 0x00, 0x00, 0x00,
  0x04, 0x00, 0x00, 0x00, 0xee, 0x3c, 0xff, 0xff, 0x38, 0x00, 0x00, 0x00,
  0x9c, 0x3c, 0xff, 0xff, 0xa0, 0x3c, 0xff, 0xff, 0xfe, 0x3c, 0xf